In [89]:
# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science

## Project Group #26: Lending Club


**Harvard University**<br/>
**Fall 2018**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader<br/>
**TF**: Jerry Peng<br/>
**Students**: Michel Atoudem Kana, William Groves, Shourya Veeraganti

<hr style="height:2pt">



SyntaxError: ignored

# Data Description and Cleaning

In [91]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

import numpy as np
import pandas as pd
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Data Description

Lending Club publishes all its historical data since its inception in 2007. It provides two online, open access datasets for accepted and rejected loans from 2007 to 2018 Q2 for a period of 11.5 years in comma-separated values (CSV) format. Each accepted loan data set has 146 features for each of observation. With basic data cleaning, we removed the index column, columns with constant value and columns associated with 90% missing values that reduced the number of features to 106. Further we identified features which are strongly correlated (r=±0.8) that left us with 82 features on the accepted loan dataset.

To help us in our goal of analyzing fairness and interpretability we downloaded the census data about American people and US economy from United States Census Bureau. We have social, economic, and geographic data from 2016 grouped by zip code provided via Piazza by CS109a instructors. This dataset has 33120 zip codes described by 135 features. Trivial columns such as name, population and others were removed that left us with 85 features. Then the census data was merged with loan accepted data on zipcode ending with 167 features. Then we further used scikit learn feature selection on SelectPercentile reducing 141 predictors for our model analysis. For reconciliation, we downloaded data for all years and then we took a random sample of 10% and saved it to our own server “https://digintu.tech/tmp/cs109a/” stored in the file “loan_accepted_10.csv” for easy access. Also removed empty/duplicate rows that left us with 200K observations.

Second dataset provided online is the rejected loan information, again from 2007 to 2018 Q2 for 11.5 years. This dataset has 9 features. Each application of rejected loan has 9 features and the total amount was worth $22million not funded. These Rejected Loans data files contain the list and details of all loan applications that did not meet Lending Club's credit underwriting policy and the application was rejected.

We group the remaining features from loan accepted merged with census data into 5 classes:
● loan data: information about the loan at the moment when it was requested
● loan follow up: information about the loan's follow up throughout its term
● borrower demographics: information about the borrower
● borrower financial profile: financial background of the borrower at the moment when he requested the loan
● borrower financial profile follow up: changes in financial profile of the borrower throughout the loan term

# Raw Data

**Helper functions**

In [92]:
"""
The following function returns the description of the features in lending club data.
Parameters
  df: dataframe containing the data
"""
def df_features_desc(df):
    df_cols_desc = pd.read_csv('https://s3.amazonaws.com/ruxton.ai/LCDataDictionary.csv')
    desc = pd.DataFrame(df.columns).merge(df_cols_desc, how='left', left_on=0, right_on='LoanStatNew')[['LoanStatNew','Description']]
    desc = pd.DataFrame(df.dtypes).reset_index().merge(desc, how='right', left_on='index', right_on='LoanStatNew')[['LoanStatNew', 0,'Description']]
    desc = desc.rename(columns={0: 'Data Type', 'LoanStatNew':'Feature'})#.sort_values(by=['Feature'])
    desc = desc.dropna()
    pd.set_option('display.max_colwidth', -1)
    desc.style.set_properties({'text-align': 'left'})
    display(HTML(desc.to_html()))

## Accepted Loans

We load all loans accepted (approved) and funded on the LendingClub marketplace from 2007 to 2018 Q3 for a period of 11.5 years from separated files into the dataframe `df_loan_accepted`.

In [154]:
#df_loan_accepted = load_data(['loan_accepted_10.csv'])
df_loan_accepted = pd.read_csv('https://s3.amazonaws.com/ruxton.ai/loans.csv')

In [155]:
df_loan_accepted.shape

(226066, 152)

There are approximately **2 millions accepted loans**, each of them has 146 columns. Many columns are however empty.

In [156]:
df_loan_accepted.describe()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,desc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,...,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,226066.000000,2.260660e+05,0.0,226066.000000,226066.000000,226066.000000,226066.000000,226066.000000,2.260660e+05,0.0,225880.000000,226061.000000,226066.000000,226066.000000,226061.000000,109951.000000,35595.000000,226061.000000,226061.000000,2.260660e+05,225898.000000,226061.000000,226066.000000,226066.000000,226066.000000,226066.000000,226066.000000,226066.000000,2.260660e+05,226066.000000,226066.000000,226066.000000,226066.000000,226066.000000,226049.000000,57582.000000,226066.0,1.198400e+04,11984.000000,226061.000000,...,218924.000000,218924.000000,218924.000000,218924.000000,220116.000000,210660.000000,218924.000000,218924.000000,218924.000000,218912.000000,218483.000000,225949.000000,226053.000000,2.189240e+05,2.210090e+05,221009.000000,2.189240e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1072.0,1072.000000,1072.0,1072.000000,842.000000,1072.000000,1072.000000,3476.000000,3476.000000,3476.000000
mean,113033.500000,8.024734e+07,NaN,15046.720427,15041.948590,15023.774244,13.073035,445.922661,7.815293e+04,NaN,18.779397,0.304303,698.673330,702.673591,0.577972,34.653591,72.486641,11.598126,0.196067,1.667220e+04,50.315296,24.108590,4200.934890,4199.986783,12096.385794,12078.246049,9515.731645,2434.780202,1.537982e+00,144.335961,24.053778,3418.043153,687.609278,675.525422,0.018633,44.227606,1.0,1.243526e+05,19.220626,0.004092,...,8.376053,8.243400,13.988229,5.570180,11.614340,0.000674,0.002823,0.083079,2.074894,94.125045,42.367071,0.126799,0.046520,1.777549e+05,5.084552e+04,23299.656887,4.357713e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,158.582211,3.0,13.677239,471.470273,11689.770737,191.878041,5035.089485,47.787825,13.233890
std,65259.777315,4.501596e+07,NaN,9190.775065,9189.602009,9193.221365,4.816208,267.446936,1.477066e+05,NaN,14.095529,0.870180,33.030786,33.031946,0.885742,21.916636,26.323355,5.645185,0.571967,2.253863e+04,24.734496,11.988146,7332.561075,7331.659935,9901.852023,9897.432270,8313.543944,2683.497635,1.207018e+01,752.966092,131.284636,6006.247777,72.929327,110.922898,0.150169,21.556635,0.0,6.764284e+04,7.809577,0.068838,...,7.325110,4.686786,8.050783,3.287111,5.648929,0.026676,0.057279,0.503315,1.827641,9.061515,36.198548,0.361658,0.378477,1.788505e+05,4.961362e+04,23129.334170,4.506476e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,133.292520,0.0,9.689210,396.520777,7590.893474,194.935844,3685.788592,6.905756,8.681392
min,1.000000,5.612100e+04,NaN,500.000000,500.000000,0.000000,5.310000,14.010000,0.000000e+00,NaN,0.000000,0.000000,640.000000,644.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.00

Below is a short description of columns in the accepted loan data set.

In [96]:
df_features_desc(df_loan_accepted)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if sys.path[0] == '':


,Feature,Data Type,Description
40,id,int64,A unique LC assigned ID for the loan listing.
41,member_id,float64,A unique LC assigned Id for the borrower member.
42,loan_amnt,int64,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
43,funded_amnt,int64,The total amount committed to that loan at that point in time.
44,funded_amnt_inv,float64,The total amount committed by investors for that loan at that point in time.
45,term,object,The number of payments on the loan. Values are in months and can be either 36 or 60.
46,int_rate,float64,Interest Rate on the loan
47,installment,float64,The monthly payment owed by the borrower if the loan originates.
48,grade,object,LC assigned loan grade
49,sub_grade,object,LC assigned loan subgrade


## Census Data
The United States Census Bureau provides data about the American people and economy. We have social, economic, and geographic data from 2016 grouped by zip code provided via Piazza by CS109a instructors.

We load census data from 2016 into the dataframe `df_census`. 


In [153]:
df_census = pd.read_csv('https://s3.amazonaws.com/ruxton.ai/zipcode_demographics_2016_USA.csv')

In [157]:
df_census.shape

(33120, 135)

There are 33120 zip codes, each of them described by 135 statistics.

In [99]:
df_census.describe()

,Unnamed: 0,Population,zip code tabulation area,owner_renter_total,owner_occupied,renter_occupied,abroad_year_ago_total,abroad_year_ago_puerto_rico,abroad_year_ago_us_islands,abroad_year_ago_foreign,households_total,mortgage_value_total,median_income_2016,male_pct,Male_0to9_Years_pct,Male_10to19_Years_pct,Male_20to29_Years_pct,Male_30to39_Years_pct,Male_40to49_Years_pct,Male_50to59_Years_pct,Male_60to69_Years_pct,Male_70p_Years_pct,female_pct,Female_0to9_Years_pct,Female_10to19_Years_pct,Female_20to29_Years_pct,Female_30to39_Years_pct,Female_40to49_Years_pct,Female_50to59_Years_pct,Female_60to69_Years_pct,Female_70p_Years_pct,White_pct,Black_pct,Native_pct,Asian_pct,Islander_pct,Other_pct,Two_race_pct,Hispanic_pct,car_to_work_pct,...,english_speaker_foreign_pct,poverty_level_below_pct,powerty_level_at_above_pct,housing_owner_occupied_pct,housing_renter_occupied_pct,rent_less_1000_pct,rent_less_1500_pct,rent_less_2000_pct,rent_less_2500_pct,rent_less_3500_pct,rent_above_3500_pct,owned_with_mortgage_pct,owner_no_mortgage_pct,mortgage_value_less_50k_pct,mortgage_value_50_100k_pct,mortgage_value_100_150k_pct,mortgage_value_150_200k_pct,mortgage_value_200_300k_pct,mortgage_value_300_500k_pct,mortgage_value_500_750k_pct,mortgage_value_750_1M_pct,mortgage_value_1Mp_pct,employment_2016_rate,worked_in_county_of_residence_2016_rate,wourked_outside_county_residence_2016_rate,less_than_15k_pct,15k_to_30k_pct,30k_to_40k_pct,40k_to_50k_pct,50_to_60k_2016_pct,60_to_75k_2016_pct,75_to_100k_2016_pct,100_to_150k_2016_pct,150_to_200k_2016_pct,over_200k_2016_pct,No_Diploma_pct,High_school_pct,Some_college_pct,Bachelors_Degree_pct,Graduate_Degree_pct
count,33120.000000,33120.000000,33120.000000,32989.000000,32989.000000,32989.000000,32989.000000,32989.000000,32989.000000,32989.000000,33120.000000,33120.000000,3.312000e+04,32799.000000,32745.000000,32745.000000,32745.000000,32745.000000,32745.000000,32745.000000,32745.000000,32745.000000,32799.000000,32703.000000,32703.000000,32703.000000,32703.000000,32703.000000,32703.000000,32703.000000,32703.000000,32799.000000,32799.000000,32799.000000,32799.000000,32799.000000,32799.000000,32799.000000,32799.000000,32571.000000,...,25962.000000,21604.000000,21604.000000,32540.000000,32540.000000,21468.000000,21468.000000,21468.000000,21468.000000,21468.000000,21468.000000,32321.000000,32321.000000,32321.000000,32321.000000,32321.000000,32321.000000,32321.000000,32321.000000,32321.000000,32321.000000,32321.000000,32799.000000,32559.000000,32559.000000,23055.000000,23055.000000,23055.000000,23055.000000,23055.000000,23055.000000,23055.000000,23055.000000,23055.000000,23055.000000,32765.000000,32765.000000,32765.000000,32765.000000,32765.000000
mean,16559.500000,9724.409300,49666.334209,9298.151808,6073.192549,3224.959259,61.003031,2.476250,0.529843,57.996938,3591.535870,1460.150906,-4.036731e+07,0.502147,0.116908,0.130183,0.122360,0.114861,0.125128,0.151852,0.129827,0.108881,0.497853,0.111313,0.122054,0.113694,0.111732,0.124679,0.153525,0.132873,0.130130,0.837472,0.076902,0.017776,0.020870,0.001072,0.022186,0.023723,0.092698,0.873221,...,0.067440,0.268171,0.731829,0.728846,0.271154,0.775819,0.122267,0.048556,0.022749,0.022636,0.007974,0.541540,0.458460,0.042717,0.104887,0.092470,0.083044,0.095607,0.075109,0.027065,0.010366,0.010274,0.811712,0.713389,0.286611,0.143071,0.189944,0.115745,0.098504,0.082846,0.098562,0.107134,0.097093,0.034172,0.032929,0.010476,0.290389,0.232441,0.148160,0.084170
std,9561.064794,14358.657599,27564.925769,13855.431955,8730.614992,6313.579477,170.145836,21.042147,5.755175,164.081808,5166.208239,2194.848451,1.591133e+08,0.067328,0.064737,0.073723,0.082944,0.065031,0.063121,0.079273,0.084550,0.088059,0.067328,0.061186,0.073770,0.078289,0.060611,0.063031,0.079532,0.087294,0.087531,0.207960,0.159127,0.092161,0.053946,0.010115,0.055872,0.034959,0.167006,0.139833,...,0.139831,0.281627,0.281627,0.175033,0.175033,0.335789,0.232051,0.147293,0.100789,0.099764,0.058117,0.179372,0.179372,0.059360,

# Loan Acceptance Data Cleaning

In this section we describe basic operations for cleaning our dataset.


In [100]:
df_loan_accepted_cleaned = df_loan_accepted.copy()

### Remove index column

In [101]:
df_loan_accepted_cleaned.drop(columns=['Unnamed: 0'], inplace=True)

### Remove columns with a constant value

In [102]:
cols_unique = list(df_loan_accepted_cleaned.columns[df_loan_accepted_cleaned.nunique()==1])
print('The following features with constant value were removed', cols_unique)
df_loan_accepted_cleaned.drop(cols_unique, axis=1, inplace=True)
print('{} features left, out of {} in the original dataset'.format(df_loan_accepted_cleaned.shape[1], df_loan_accepted.shape[1]))

The following features with constant value were removed ['policy_code', 'hardship_type', 'deferral_term', 'hardship_length']
147 features left, out of 152 in the original dataset


### Remove columns associated with over 90% missing values

In [103]:
df_missing = (df_loan_accepted_cleaned.isnull().sum()/df_loan_accepted_cleaned.shape[0]).to_frame('perc_missing').reset_index()
cols_missing = list(df_missing[df_missing.perc_missing>0.9]['index'])
print('The following features with over 90% missing values were removed.')
print(df_missing[df_missing.perc_missing>0.9])
df_loan_accepted_cleaned.drop(cols_missing, axis=1, inplace=True)
print('')
print('{} features left, out of {} in the original dataset'.format(df_loan_accepted_cleaned.shape[1], df_loan_accepted.shape[1]))

The following features with over 90% missing values were removed.
                                          index  perc_missing
1    member_id                                   1.000000    
19   desc                                        1.000000    
56   annual_inc_joint                            0.946989    
57   dti_joint                                   0.946989    
58   verification_status_joint                   0.949205    
114  revol_bal_joint                             1.000000    
115  sec_app_fico_range_low                      1.000000    
116  sec_app_fico_range_high                     1.000000    
117  sec_app_earliest_cr_line                    1.000000    
118  sec_app_inq_last_6mths                      1.000000    
119  sec_app_mort_acc                            1.000000    
120  sec_app_open_acc                            1.000000    
121  sec_app_revol_util                          1.000000    
122  sec_app_open_act_il                         1.000000    
123 

### Remove duplicate rows

In [104]:
orig_rows_count = df_loan_accepted_cleaned.shape[0]
df_loan_accepted_cleaned.drop_duplicates(inplace=True)
print("{} duplicated rows were removed.".format(orig_rows_count-df_loan_accepted_cleaned.shape[0]))

0 duplicated rows were removed.


### Remove empty rows

In [105]:
orig_rows_count = df_loan_accepted_cleaned.shape[0]
df_loan_accepted_cleaned.dropna(inplace=True, how='all')
print("{} empty rows were removed.".format(orig_rows_count-df_loan_accepted_cleaned.shape[0]))

0 empty rows were removed.


# More Data Cleaning

## Variables Groups

We divide the columns in the following groups:

- **loan data**: information about the loan at the moment when it was requested
- **loan followup**: information about the loan's followup throughout its term
- **borrower demographics**: information about the borrower
- **borrower financial profile**: financial background of the borrower at the moment when he requested the loan
- **borrower financial profile followup**: changes in financial profile of the borrower throughout the loan term

In [106]:
cols_loan_data = ['loan_amnt','funded_amnt','funded_amnt_inv','term','int_rate','installment','grade','sub_grade','issue_d','loan_status','purpose','title','initial_list_status','application_type','disbursement_method']
cols_loan_followup = ['out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','next_pymnt_d','debt_settlement_flag','pymnt_plan']
cols_borrower_demographics = ['emp_title','emp_length','home_ownership','annual_inc','verification_status', 'zip_code','addr_state']
cols_borrower_finance_profile = ['dti','delinq_2yrs','earliest_cr_line','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec','revol_bal','revol_util','total_acc','tot_coll_amt','mort_acc','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit','hardship_flag']
cols_borrower_finance_profile_followup = ['last_credit_pull_d','collections_12_mths_ex_med','mths_since_last_major_derog','acc_now_delinq','tot_cur_bal','open_acc_6m','open_act_il','open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy','bc_util','chargeoff_within_12_mths','delinq_amnt','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mths_since_recent_bc','mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m']

## Loan Data

In this section we manually clean-up come features describing the resquest of loan by borrowers.

**Issue date**

Let's convert the loan's issue date into a datatime type and add month and quarter.

In [107]:
df_loan_accepted_cleaned['issue_q'] = pd.to_datetime(df_loan_accepted_cleaned.issue_d, format='%b-%Y').dt.to_period('Q')
df_loan_accepted_cleaned['issue_m'] = df_loan_accepted_cleaned.issue_d.str.replace(r'-\d+', '')

**Loan status**

Delinquency happens when a borrower fails to pay the minimum amount for an outstanding debt. In the countplot below we can see the amount of loans that incurred in any stage of delinquency, according to the definitions used by Lending Club.

    Charged Off — defaulted loans for which there is no expectation from the lender in recovering the debt
    Default — borrower has failed to pay his obligations for more than 120 days
    Late — borrower has failed to pay his obligations for 31 to 120 days
    Grace Period — borrower still has time to pay his obligations without being considered delinquent
    Late — payment is late by 16 to 30 days
    
The count of loans within each stage is given below.

In [108]:
df_loan_accepted_cleaned.loan_status.value_counts()

Fully Paid                                             107737
Current                                                87796 
Charged Off                                            26761 
Late (31-120 days)                                     2210  
In Grace Period                                        843   
Late (16-30 days)                                      442   
Does not meet the credit policy. Status:Fully Paid     213   
Does not meet the credit policy. Status:Charged Off    63    
Default                                                1     
Name: loan_status, dtype: int64

Very few old loans have the status 'Does not meet the credit policy' and will not be considered in our project.

In [109]:
df_loan_accepted_cleaned.drop(df_loan_accepted_cleaned[df_loan_accepted_cleaned.loan_status=='Does not meet the credit policy. Status:Fully Paid'].index, inplace=True)
df_loan_accepted_cleaned.drop(df_loan_accepted_cleaned[df_loan_accepted_cleaned.loan_status=='Does not meet the credit policy. Status:Charged Off'].index, inplace=True)

We add a new feature for successfully paid loans.

In [110]:
df_loan_accepted_cleaned['success'] = df_loan_accepted_cleaned['loan_status']
df_loan_accepted_cleaned.replace({'success':{'Charged Off': 0,
                                             'Fully Paid': 1, 
                                             'Current': 2, 
                                             'In Grace Period': 3,
                                             'Late (16-30 days)': 4,
                                             'Late (31-120 days)': 5,
                                             'Default': 6 }}, inplace=True)
df_loan_accepted_cleaned['success'] = df_loan_accepted_cleaned['success'].astype('int')

**Term and interest rate**

We turn the term and interest rate into numbers.

In [111]:
df_loan_accepted_cleaned.term.unique()

array(['36 months', '60 months'], dtype=object)

In [112]:
df_loan_accepted_cleaned.term.replace(' 36 months', 36, inplace=True)
df_loan_accepted_cleaned.term.replace(' 60 months', 60, inplace=True)
#df_loan_accepted_cleaned.term = df_loan_accepted_cleaned.term.astype('int')

In [113]:
df_loan_accepted_cleaned.term.unique()

array(['36 months', '60 months'], dtype=object)

In [114]:
df_loan_accepted_cleaned.int_rate.head()

0    13.59
1    17.97
2    13.59
3    12.74
4    19.99
Name: int_rate, dtype: float64

In [115]:
#df_loan_accepted_cleaned.int_rate = df_loan_accepted_cleaned.int_rate.str[:-1]
df_loan_accepted_cleaned.int_rate = df_loan_accepted_cleaned.int_rate.astype('float32')

In [116]:
df_loan_accepted_cleaned.int_rate.head()

0    13.590000
1    17.969999
2    13.590000
3    12.740000
4    19.990000
Name: int_rate, dtype: float32

**Loan amounts**

We will transform the amounts into integers

In [117]:
df_loan_accepted_cleaned.loan_amnt = df_loan_accepted_cleaned.loan_amnt.astype('int')

We will drop rows where `loan_amnt`, `funded_amnt`, `funded_amnt_inv` or `installment` is missing. We transform the amounts into integers.

In [118]:
df_loan_accepted_cleaned = df_loan_accepted_cleaned[df_loan_accepted_cleaned.loan_amnt.notnull() &
                                                    df_loan_accepted_cleaned.funded_amnt.notnull() & 
                                                    df_loan_accepted_cleaned.funded_amnt_inv.notnull() & 
                                                    df_loan_accepted_cleaned.installment.notnull()]

In [119]:
df_loan_accepted_cleaned.funded_amnt_inv = df_loan_accepted_cleaned.loan_amnt.astype('int')

**Title**

We will replace missing `title` values with NA for not available.

In [120]:
df_loan_accepted_cleaned.replace({'title': {np.nan: 'N/A'}}, inplace=True)

There are thousands of distinct titles entered by borrowers for their loan.

In [121]:
df_loan_accepted_cleaned.title.unique().shape

(8526,)

 We will prepare the titles for natural language features extration.

In [122]:
def clean_text(text):
    #https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/
    # lower case
    text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
    # remove punctuation
    text = text.str.replace('[^\w\s]','') 
    # remove stop words
    from nltk.corpus import stopwords
    stop = stopwords.words('english')
    text = text.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    # correct spelling
    #from textblob import TextBlob
    #text = text.apply(lambda x: str(TextBlob(x).correct()))
    # lemmatization 
    from textblob import Word
    text = text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    return text

In [123]:
df_loan_accepted_cleaned.title = clean_text(df_loan_accepted_cleaned.title)

In [124]:
df_loan_accepted_cleaned.title.unique()

array(['debt consolidation', 'credit card refinancing', 'business', ...,
       'pay credit card medical bill', 'better future',
       'well fargo payoff'], dtype=object)

**Miscellenous**

The columns `initial_list_status`,`application_type`,`disbursement_method` describe two categories each, one of them being largely dominant.

In [125]:
df_loan_accepted_cleaned.initial_list_status.value_counts()

w    153564
f    72226 
Name: initial_list_status, dtype: int64

In [126]:
df_loan_accepted_cleaned.application_type.value_counts()

Individual    213806
Joint App     11984 
Name: application_type, dtype: int64

In [127]:
df_loan_accepted_cleaned.disbursement_method.value_counts()

Cash         218059
DirectPay    7731  
Name: disbursement_method, dtype: int64

We remove those columns.

In [128]:
df_loan_accepted_cleaned.drop(columns=['initial_list_status','application_type','disbursement_method'], inplace=True)

## Loan Followup

In this section we manually clean-up the information about the current status of an active loan. These columns tell for example how much was paid back, when the next payment is to be expected etc.

For the goal of our project, we will consider `total_rec_late_fee` which indicate that the borrower had issues with paying the installment at some point in the past. This information could help computing the probability of charge-off or default for ongoing loans. 

We will also consider features such as `last_pymnt_d`, `total_rec_prncp`, `total_rec_int`, `last_pymnt_d` for computing the return of investment for closed loans.

We delete the remaining follow-up columns.

In [129]:
df_loan_accepted_cleaned.drop(columns=['out_prncp','out_prncp_inv','debt_settlement_flag','pymnt_plan','recoveries','hardship_flag'], inplace=True)

## Borrower demographics

In this section we manually clean-up data related to the borrower.

**Employment Title**

We replace missing employment title by 'N/A'. 

In [130]:
df_loan_accepted_cleaned.replace({'emp_title': {np.nan: 'N/A'}}, inplace=True)

**Employment Length**

Missing employment length is replaced by 0.

In [131]:
df_loan_accepted_cleaned.emp_length.fillna(value=0,inplace=True)
df_loan_accepted_cleaned.emp_length.replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df_loan_accepted_cleaned.emp_length.replace(to_replace='self-employed', value='0', inplace=True, regex=True)
df_loan_accepted_cleaned.emp_length.replace(to_replace='', value='0', inplace=True, regex=True)
df_loan_accepted_cleaned.emp_length = df_loan_accepted_cleaned.emp_length.astype(int)

**Annual Income**

There are too many outliers in `annual_inc`, which should be removed.

In [132]:
orig_rows_count = df_loan_accepted_cleaned.shape[0]
df_loan_accepted_cleaned = df_loan_accepted_cleaned[~(df_loan_accepted_cleaned.annual_inc > 250000)]
print("{} rows removed with annual_inc > 250000.".format(orig_rows_count-df_loan_accepted_cleaned.shape[0]))

2575 rows removed with annual_inc > 250000.


## Borrower Financial Profile

In this section we clean columns which describe the credit history of the borrower.

**Revolving Line Utilization Rate**

We remove the '%' sign from the revolving line utilization rate and turn the column to float datatype.

In [133]:
df_loan_accepted_cleaned.revol_util = df_loan_accepted_cleaned.revol_util.fillna('0%')
df_loan_accepted_cleaned.revol_util = df_loan_accepted_cleaned.revol_util.str[:,-1]
df_loan_accepted_cleaned.revol_util = df_loan_accepted_cleaned.revol_util.astype('float32')

**Credit History**

It is safe to impute missing values for the following columns with zero.

In [134]:
cols = ['dti','delinq_2yrs','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec','revol_bal','revol_util','total_acc','tot_coll_amt','mort_acc','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit']

df_loan_accepted_cleaned[cols] = df_loan_accepted_cleaned[cols].fillna(0)

**Trade Delinquency**

The percent of trades never delinquent `pct_tl_nvr_dlq` is set to 100% if it is missing.

In [135]:
df_loan_accepted_cleaned.pct_tl_nvr_dlq.fillna(100, inplace=True)

## Borrower Financial Profile Followup

In this section we manually clean some columns which contain a more current financial information about the borrower.

`last_credit_pull_d` indicates how old the financial information about the borrower is. It is safe to drop this feature.

In [136]:
df_loan_accepted_cleaned.drop(columns=['last_credit_pull_d'], inplace=True)

We will set the ratio of total current balance to high credit/credit limit for all bankcard accounts to 100% when missing.

In [137]:
df_loan_accepted_cleaned.bc_util.fillna(100, inplace=True)

It is safe to impute missing values for the remaining columns with zero.

In [138]:
cols = ['collections_12_mths_ex_med','mths_since_last_major_derog','acc_now_delinq','tot_cur_bal','open_acc_6m','open_act_il','open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy','chargeoff_within_12_mths','delinq_amnt','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mths_since_recent_bc','mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m']

df_loan_accepted_cleaned[cols] = df_loan_accepted_cleaned[cols].fillna(0)

## Correlation

In this section, we check the correlation between all remaining features.

In [139]:
def find_high_correlated_features(frame):
    new_corr = frame.corr()
    new_corr.loc[:,:] = np.tril(new_corr, k=-1) 
    new_corr = new_corr.stack()
    print(new_corr[(new_corr > 0.8) | (new_corr < -0.8)])


In [140]:
find_high_correlated_features(df_loan_accepted_cleaned)   

funded_amnt                 loan_amnt                 0.999794
funded_amnt_inv             loan_amnt                 1.000000
                            funded_amnt               0.999794
installment                 loan_amnt                 0.945700
                            funded_amnt               0.946007
                            funded_amnt_inv           0.945700
fico_range_high             fico_range_low            1.000000
total_pymnt_inv             total_pymnt               0.999579
total_rec_prncp             total_pymnt               0.965924
                            total_pymnt_inv           0.965485
last_fico_range_low         last_fico_range_high      0.845682
open_rv_24m                 open_rv_12m               0.830432
all_util                    il_util                   0.811630
avg_cur_bal                 tot_cur_bal               0.825900
num_actv_rev_tl             num_actv_bc_tl            0.833767
num_bc_sats                 num_actv_bc_tl            0

As shown above, `installment` carries the same information as the `funded_amnt_in`. Similarly `open_il_24m` and `open_il_12m` are highly correlated. Same for `open_rv_24m` and `open_rv_12m`; `all_util` and `il_util`; `total_rev_hi_lim` and `revol_bal`; `bc_util` and `revol_util`; `avg_cur_bal` and `tot_cur_bal`; `num_actv_bc_tl` and `num_actv_bc_tl`; `num_tl_30dpd` and `acc_now_delinq`. We consider dropping some of those columns below.

In [141]:
df_loan_accepted_cleaned.drop(columns=['open_il_12m','open_rv_12m','il_util','revol_bal', 
                                       'revol_util','avg_cur_bal','num_actv_bc_tl','num_tl_30dpd'], inplace=True)

## Census Data

After cleaning the loan acceptance data, we will now clean and add census data to it.

### Cleaning census data

In [142]:
df_census_cleaned = df_census.copy()

We keep the following columns and remove the rest from the census data.

In [143]:
census_cols = ['Population', 'zip code tabulation area', 'median_income_2016', 
               'male_pct', 'female_pct', 
               'Black_pct', 'Native_pct', 'Asian_pct', 'Hispanic_pct', 
               'household_family_pct', 'poverty_level_below_pct', 'Graduate_Degree_pct', 'employment_2016_rate']
df_census_cleaned = df_census_cleaned[census_cols]

We remove rows with median income less than zero.

In [144]:
df_census_cleaned = df_census_cleaned[df_census_cleaned.median_income_2016>0]

We first fill missing values with zeros in the census dataset.

In [145]:
df_census_cleaned.isnull().sum()

Population                  0   
zip code tabulation area    0   
median_income_2016          0   
male_pct                    0   
female_pct                  0   
Black_pct                   0   
Native_pct                  0   
Asian_pct                   0   
Hispanic_pct                0   
household_family_pct        0   
poverty_level_below_pct     9760
Graduate_Degree_pct         0   
employment_2016_rate        0   
dtype: int64

In [146]:
df_census_cleaned.fillna(0, inplace=True)

Add a new column with zip codes in the format 123XX

In [147]:
df_census_cleaned['zip_code'] = df_census_cleaned['zip code tabulation area'].astype('str')
df_census_cleaned['zip_code'] = df_census_cleaned['zip_code'].str.pad(5, 'left', '0')
df_census_cleaned['zip_code'] = df_census_cleaned['zip_code'].str.slice(0,3)
df_census_cleaned['zip_code'] = df_census_cleaned['zip_code'].str.pad(5, 'right', 'x')
df_census_cleaned.drop(columns=['zip code tabulation area'], inplace=True)

Aggregate by zipcode and take the mean of census values

In [148]:
df_census_cleaned = df_census_cleaned.groupby(['zip_code']).mean().reset_index()

In [149]:
df_census_cleaned.tail()

,zip_code,Population,median_income_2016,male_pct,female_pct,Black_pct,Native_pct,Asian_pct,Hispanic_pct,household_family_pct,poverty_level_below_pct,Graduate_Degree_pct,employment_2016_rate
884,995xx,6268.461538,59252.903846,0.526823,0.473177,0.020715,0.459778,0.040214,0.049570,0.684839,0.063769,0.069403,0.760864
885,996xx,2560.727273,50849.181818,0.534283,0.465717,0.007251,0.565686,0.034245,0.027964,0.695934,0.089168,0.046758,0.749790
886,997xx,2019.910448,48305.447761,0.546723,0.453277,0.007017,0.652971,0.010724,0.022896,0.697308,0.029398,0.047139,0.733550
887,998xx,3639.714286,61540.857143,0.518490,0.481510,0.015683,0.196095,0.031797,0.044007,0.663403,0.036125,0.076276,0.829295
888,999xx,2420.888889,49593.444444,0.532552,0.467448,0.001796,0.285796,0.015533,0.024939,0.558452,0.123077,0.061948,0.822142


### Loan and census data consolidation

In [150]:
 df_loan_accepted_census_cleaned = pd.merge(df_loan_accepted_cleaned, df_census_cleaned, on='zip_code')

In [151]:
print('The merged loan and census dataset has {} features'.format(df_loan_accepted_census_cleaned.shape[1]))

The merged loan and census dataset has 109 features


Let's save the final accepted loan dataset augmented with census data to disk.

Merge accepted loan data with census data using the zip code

In [152]:
df_loan_accepted_census_cleaned.to_csv('df_loan_accepted_census_cleaned.csv')